## Retriever Fusion

使用 LangChain 的 EnsembleRetriever 组合 BM25 和 Chroma 两种检索方法，从而在检索过程中结合关键词匹配和语义相似性搜索的优势。通过这种组合，我们能够在查询时获得更全面的结果。

In [21]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from chat_model_client import get_model
from pprint import pprint

1. 加载页面

In [22]:
loader = WebBaseLoader("https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a")
docs = loader.load()
spliter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
chunks = spliter.split_documents(docs)
query = "马拉松的经济价值"

2. BM25 向量检索器

BM25 是一种基于词频和逆文档频率（TF-IDF）的传统检索算法，属于稀疏匹配

In [23]:
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 2 # 设置 BM25 检索器返回的文档数量
bm25_retriever.get_relevant_documents(query)

[Document(metadata={'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端', 'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力----新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。45万+的报名人数虽属罕见，但整个2024年，动辄20万+的报名数字在国内头部马拉松赛事中已屡见不鲜。 2014年，《国务院关于加快发展体育产业促进体育消费的若干意见》颁布，自此，以马拉松为代表的中国路跑运动迎来黄金十年，不断倍增的报名人数成为“跑马热”最直观的印证。“周末两眼一睁就是马拉松，朋友圈被越来越多去全国各地参赛的跑友‘刷屏’了！”来自北京的跑者王鹏表示。 中国马拉松官网数据显示，2024年，全国共举办路跑赛事671场，参赛人次约656万。赛事分布范围涵盖了全国31个省级行政区，261个市，537个区县。仅2024年11月，国内就有114场赛事鸣枪举办，其中1', 'language': 'zh'}, page_content='（责任编辑 贾紫来）\n\n\n\n\n精彩推荐\n\n\n\n\n\n\n新华网政企服务平台特邀入驻\n\n\n\n\n\n\n“我为群众办实事”网络平台\n\n\n\n\n\n\n法律说吧｜聚焦中小企业合规发展\n\n\n\n\n\n\n暖视频：总有感动在身边'),
 Document(metadata={'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端', 'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力

3. Chroma 向量存储检索器

使用Chroma 向量存储绑定Retriever，属于密集匹配

In [24]:
vs = Chroma.from_documents(chunks, OllamaEmbeddings(model = "llama2-chinese"))
ch_retriever = vs.as_retriever()
ch_retriever.get_relevant_documents(query)

[Document(metadata={'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力----新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。45万+的报名人数虽属罕见，但整个2024年，动辄20万+的报名数字在国内头部马拉松赛事中已屡见不鲜。 2014年，《国务院关于加快发展体育产业促进体育消费的若干意见》颁布，自此，以马拉松为代表的中国路跑运动迎来黄金十年，不断倍增的报名人数成为“跑马热”最直观的印证。“周末两眼一睁就是马拉松，朋友圈被越来越多去全国各地参赛的跑友‘刷屏’了！”来自北京的跑者王鹏表示。 中国马拉松官网数据显示，2024年，全国共举办路跑赛事671场，参赛人次约656万。赛事分布范围涵盖了全国31个省级行政区，261个市，537个区县。仅2024年11月，国内就有114场赛事鸣枪举办，其中1', 'language': 'zh', 'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端'}, page_content='，描绘出了活力中国的精彩画卷，也汇聚起了体育强国建设道路上笃行致远的坚实跬步。（刁文静对此文亦有贡献）'),
 Document(metadata={'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力----新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。45万+的报名人数虽属罕见，但整个2024年，动辄20万+的报名数字在国内头部马拉松赛事中已屡见不鲜。 2014年，《国务院关于加快发展体育产业促进体育消费的若干意见》颁布，自此，以马拉松为代表的中国路跑运动迎来黄金十年，不断倍

4. 融合向量检索

可以使用多个检索器，并且配置他们的融合权重

In [26]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, ch_retriever], weights=[0.7, 0.3]
)
ensemble_retriever.get_relevant_documents("马拉松的经济价值")


[Document(metadata={'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端', 'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力----新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。45万+的报名人数虽属罕见，但整个2024年，动辄20万+的报名数字在国内头部马拉松赛事中已屡见不鲜。 2014年，《国务院关于加快发展体育产业促进体育消费的若干意见》颁布，自此，以马拉松为代表的中国路跑运动迎来黄金十年，不断倍增的报名人数成为“跑马热”最直观的印证。“周末两眼一睁就是马拉松，朋友圈被越来越多去全国各地参赛的跑友‘刷屏’了！”来自北京的跑者王鹏表示。 中国马拉松官网数据显示，2024年，全国共举办路跑赛事671场，参赛人次约656万。赛事分布范围涵盖了全国31个省级行政区，261个市，537个区县。仅2024年11月，国内就有114场赛事鸣枪举办，其中1', 'language': 'zh'}, page_content='（责任编辑 贾紫来）\n\n\n\n\n精彩推荐\n\n\n\n\n\n\n新华网政企服务平台特邀入驻\n\n\n\n\n\n\n“我为群众办实事”网络平台\n\n\n\n\n\n\n法律说吧｜聚焦中小企业合规发展\n\n\n\n\n\n\n暖视频：总有感动在身边'),
 Document(metadata={'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端', 'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力